In [1]:
import os
from os.path import isfile
import pandas as pd
import numpy as np
from scipy.stats import norm
import nibabel as nb
import math as m
import pickle
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from compile_studies import compile_studies
from tfce import tfce
from scipy.io import loadmat
from simulate_noise import simulate_noise

cwd = os.getcwd()
raw_folder = cwd + '/DataRaw/'
pickle_folder = cwd + '/DataPickle/'
mask_folder = cwd + '/MaskenEtc/'

'''filename = askopenfilename()
df = pd.read_excel(filename, engine='openpyxl', header=None)'''

df = pd.read_excel(raw_folder + 'Metastocalculate.xlsx', engine='openpyxl', header=None)
with open(pickle_folder + 'experiments.pickle', 'rb') as f:
    experiments, tasks = pickle.load(f)

In [2]:
for index, row in df.iterrows():
    if row[0] == 'M': #Main Effect Analysis
        s0 = compile_studies(df, index, experiments, tasks)
        if len(s0) >= 12:
            category = df.iloc[index, 1]
            mean_subjects = experiments.iloc[s0].Subjects.mean()
            print(category + ' : ' + str(len(s0)) + 'experiments; average of {mean} subjects per experiment'.format(mean=mean_subjects))

Other : 19experiments; average of 22.263157894736842 subjects per experiment
Cwclassic : 37experiments; average of 25.756756756756758 subjects per experiment
Cwvariant : 21experiments; average of 26.333333333333332 subjects per experiment
CwIvsC : 58experiments; average of 25.96551724137931 subjects per experiment
CwIvsN : 41experiments; average of 22.21951219512195 subjects per experiment
CwIvsNwords : 17experiments; average of 22.235294117647058 subjects per experiment
CwIvsNsymbolsletters : 23experiments; average of 22.304347826086957 subjects per experiment


### ComputeALEtfce

In [3]:
s0 = compile_studies(df, 0, experiments, tasks)
experiments = experiments.loc[s0]
study = 'Other'

template = nb.load(mask_folder + "Grey10.nii")
template_data = template.get_fdata()
template_shape = template_data.shape
pad_tmp_shape = [value+30 for value in template_shape]


prior = np.zeros(template_shape, dtype=bool)
prior[template_data > 0.1] = 1
uc = 0.001
c = str(uc)[2:]
eps = np.finfo(float).eps

num_exp = len(s0)
mb = 1
for i in s0:
    mb = mb*(1-np.max(experiments.at[i, 'Kernel']))

bin_edge = np.arange(0.00005,1-mb+0.001,0.0001)
bin_center = np.arange(0,1-mb+0.001,0.0001)
step = 1/0.0001

folders_req = ['Volumes', 'NullDistributions', 'VolumesZ', 'VolumesTFCE', 'Results', 'Images', 'Foci']
folders_req_imgs = ['Foci', 'ALE', 'TFCE']

try:
    os.mkdir('ALE')
    for folder in folders_req:
        os.mkdir('ALE/' + folder)
        if folder == 'Images':
            for sub_folder in folders_req_imgs:
                os.mkdir('ALE/Images/' + sub_folder)
except FileExistsError:
    pass

if isfile(cwd + '/ALE/Foci/' + study + '.nii'):
    print('{} - loading Foci'.format(study))
    foci_arr = nb.load(cwd + '/ALE/Foci/' + study + '.nii').get_fdata()
else:
    print('{} - illustrate Foci'.format(study))
    
    foci_arr = np.zeros(template_shape)
    nested_list = [experiments.XYZ[i].T[:,:3].tolist() for i in s0]
    flat_list = np.array([item for sublist in nested_list for item in sublist])
    foci_arr[tuple(flat_list.T)] += 1
    foci_arr[~prior] = np.nan 
    ni_img = nb.Nifti1Image(foci_arr, template.affine)
    nb.save(ni_img, cwd + '/ALE/Foci/' + study + '.nii')
    


if isfile(cwd + '/ALE/NullDistributions/' + study + '.pickle'):
    print('{} - loading ALE'.format(study))
    print('{} - loading null PDF'.format(study))
    ale_arr = nb.load(cwd + '/ALE/Volumes/' + study + '.nii').get_fdata()
    ale_arr = np.nan_to_num(ale_arr)
    with open(cwd + '/ALE/NullDistributions/' + study + '.pickle', 'rb') as f:
            ale_hist, last_used, c_null = pickle.load(f)
else:
    print('{} - computing ALE'.format(study))
    
    ale_arr = np.ones(template_shape)
    hx = np.zeros((len(s0),len(bin_edge)))
    for c, i in enumerate(s0):
        data = np.zeros(pad_tmp_shape)
        for ii in range(experiments.at[i, 'Peaks']):
            coords = experiments.XYZ[i].T[:,:3][ii]
            x_range = (coords[0],coords[0]+31)
            y_range = (coords[1],coords[1]+31)
            z_range = (coords[2],coords[2]+31)
            data[x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]] = \
            np.maximum(data[x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]],
                       experiments.at[i, 'Kernel'])
        data = data[15:data.shape[0]-15,15:data.shape[1]-15, 15:data.shape[2]-15]
        bin_idxs, counts = np.unique(np.digitize(data[prior], bin_edge),return_counts=True)
        hx[c,bin_idxs] = counts
        ale_arr = np.multiply(ale_arr, 1-data)

    ale_arr = 1-ale_arr
    ale_arr[~prior] = np.nan
    
    #Save ALE scores in Nifti
    ni_img = nb.Nifti1Image(ale_arr, template.affine)
    nb.save(ni_img, cwd + '/ALE/Volumes/' + study + '.nii')

    
    print('{} - permutation-null PDF'.format(study))
    step = 1/np.mean(np.diff(bin_center))
    ale_hist = hx[0,:]
    for i in range(1,len(s0)):
        v1 = ale_hist
        v2 = hx[i,:]

        da1 = np.where(v1 > 0)[0]
        da2 = np.where(v2 > 0)[0]

        v1 = ale_hist/np.sum(v1)
        v2 = hx[i,:]/np.sum(v2)

        ale_hist = np.zeros((len(bin_center),))
        for i in range(len(da2)):
            p = v2[da2[i]]*v1[da1]
            score = 1-(1-bin_center[da2[i]])*(1-bin_center[da1])
            ale_bin = np.round(score*step).astype(int)
            ale_hist[ale_bin] = np.add(ale_hist[ale_bin], p)

    last_used = np.where(ale_hist>0)[0][-1]
    c_null = np.flip(np.cumsum(np.flip(ale_hist[:last_used+1])))

    pickle_object = (ale_hist, last_used, c_null)
    with open(cwd + '/ALE/NullDistributions/' + study + '.pickle', "wb") as f:
        pickle.dump(pickle_object, f)

if isfile(cwd + '/ALE/VolumesTFCE/' + study + '.nii'):
    print('{} - loading p-values'.format(study))
    
    z_arr = nb.load(cwd + '/ALE/VolumesZ/' + study + '.nii').get_fdata()
    z_arr = np.nan_to_num(z_arr)
    
    tfce_arr = nb.load(cwd + '/ALE/VolumesTFCE/' + study + '.nii').get_fdata()
else:
    print('{} - computing p-values'.format(study))
    
    ale_step = np.round(ale_arr*step)
    palette, index = np.unique(ale_step, return_inverse=True)
    index = palette[index].astype(int)
    p = c_null[index].reshape(ale_step.shape)
    p[p < eps] = eps
    z_arr = norm.ppf(1-p)
    
    tfce_arr = tfce(invol=z_arr, voxel_dims=template.header.get_zooms())
    tfce_arr[~prior] = np.nan
    
    tfce_img = nb.Nifti1Image(tfce_arr, template.affine)
    nb.save(tfce_img, cwd + '/ALE/VolumesTFCE/' + study + '.nii')
    
    z_arr[~prior] = np.nan
    z_img = nb.Nifti1Image(z_arr, template.affine)
    nb.save(z_img, cwd + '/ALE/VolumesZ/' + study + '.nii')
    

Other - loading Foci
Other - loading ALE
Other - loading null PDF
Other - loading p-values


In [4]:
num_peaks = experiments.loc[:,'Peaks']
kernels = experiments.loc[:,'Kernel']

from scipy.io import loadmat
permSpace5 = loadmat("MaskenEtc/permSpace5.mat")

sample_space = permSpace5["allXYZ"]
delta_t = np.max(z_arr)/100

In [5]:
for i in range(1000):
    simulate_noise(sample_space = sample_space,
                   s0 = s0,
                   num_peaks = num_peaks,
                   kernels = kernels,
                   c_null = c_null,
                   tfce_params = [delta_t, 0.6, 2])

time taken for ale calc:0.1407351699999999
time taken for p-value calc:0.043309320000000096
time taken for tfce calc:3.028588858
time taken for ale calc:0.09849789400000031
time taken for p-value calc:0.03796099099999939
time taken for tfce calc:2.992041489
time taken for ale calc:0.09863261599999973
time taken for p-value calc:0.03730211400000005
time taken for tfce calc:3.0731898189999995
time taken for ale calc:0.0996185900000004
time taken for p-value calc:0.0386036359999995


KeyboardInterrupt: 